In [ ]:
import os
import sys
import subprocess
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import time

In [ ]:
ss=0.0005
ts=0.0001
k=31.0
calcno=1234567

dataframe = pd.read_excel("input_from_gips.xlsx")
dataframe['C'][0]

with open('input_skeleton.txt') as inp_skel:
    all_lines = inp_skel.readlines()

inp = open("input.txt","w")

print(all_lines[0], end = "",flush =True,file=inp)
print(all_lines[1], end = "",flush =True,file=inp)
print(all_lines[2], end = "",flush =True,file=inp)
print(int(calcno), end = "\n",flush =True,file=inp)
print(all_lines[3], end = "",flush =True,file=inp)
print(dataframe['C'][1]/1000," ",dataframe['C'][2]/1000,file=inp)
print(all_lines[4], end = "",flush =True,file=inp)
print(dataframe['C'][3],file=inp)

print(all_lines[5], end = "",flush =True,file=inp)
print("20", end = "\n",flush =True,file=inp)
print(all_lines[6], end = "",flush =True,file=inp)
print("5", end = "\n",flush =True,file=inp)

print(all_lines[7], end = "",flush =True,file=inp)
print(int(dataframe['C'][4]),file=inp)

print(all_lines[8], end = "",flush =True,file=inp)
print(dataframe['C'][5],file=inp)

print(all_lines[9], end = "",flush =True,file=inp)
print(dataframe['C'][6]," ",dataframe['C'][7],file=inp)

print(all_lines[10], end = "",flush =True,file=inp)
print(dataframe['C'][8]," ",dataframe['C'][9],file=inp)

print(all_lines[11], end = "",flush =True,file=inp)
print(dataframe['C'][10]," ",dataframe['C'][11],file=inp)

print(all_lines[12], end = "",flush =True,file=inp)
print(dataframe['C'][12]," ",dataframe['C'][13],file=inp)

print(all_lines[13], end = "",flush =True,file=inp)
print(dataframe['C'][14],file=inp)

print(all_lines[14], end = "",flush =True,file=inp)
print(dataframe['C'][15],file=inp)

print(all_lines[15], end = "",flush =True,file=inp)
print(dataframe['C'][16],file=inp)

print(all_lines[16], end = "",flush =True,file=inp)
print(dataframe['C'][17],file=inp)

print(all_lines[17], end = "",flush =True,file=inp)
print(k, end = "\n",flush =True,file=inp)

print(all_lines[18], end = "",flush =True,file=inp)
print(ss, end = "\n",flush =True,file=inp)
print(all_lines[19], end = "",flush =True,file=inp)
print(ts, end = "\n",flush =True,file=inp)

print(all_lines[20], end = "",flush =True,file=inp)
print(all_lines[21], end = "",flush =True,file=inp)
print(all_lines[22], end = "",flush =True,file=inp)
print(all_lines[23], end = "",flush =True,file=inp)
print(all_lines[24], end = "",flush =True,file=inp)


inp.close()

In [ ]:
if os.path.exists('OUTPUT1.DAT') and os.path.exists('TUBEWALLTEMP.QTP'):
    os.remove('OUTPUT1.DAT')
    os.remove('TUBEWALLTEMP.QTP')

In [ ]:
'''
the code below will execute and produce output txt
'''
c = "whb.exe"
def execute(command):
    popen = subprocess.Popen(command, stdout=subprocess.PIPE)
    lines_iterator = iter(popen.stdout.readline, b"")
    file1 = open("output.txt","w")
    while popen.poll() is None:
        for line in lines_iterator:
            nline = line.rstrip()
            print(nline.decode("latin"), end = "\r\n",flush =True, file=file1) # yield line
    file1.close()
execute(c)

In [ ]:
with open('TEMPERATUREPROFILE.TXT') as f:
    lines = f.readlines()

data = np.array(lines[2:])
h = np.vectorize(lambda s: s.strip())(data)

l=[]
t=[]
for i in h:
    j = i.split('        ')
    l.append(float(j[0]))
    t.append(float(j[1]))

len = np.array(l, dtype=float)
temp = np.array(t, dtype=float)

In [ ]:
count = -1
for i in temp:
    if i>=400:
        count = count + 1
count = count + 1
if count>0:
    print("we need coating upto ",len[count])
else:
    print("we dont need any coating") 


In [ ]:
plt.plot(len, temp)

plt.title("Temperature profile graph")
plt.xlabel("Tube length -> m")
plt.ylabel("Temperature -> Deg C")

plt.show()

In [ ]:
figure, axis = plt.subplots(2, 2)
axis[0,0].plot(len, temp)

axis[0,0].set_title("Temperature profile graph at clean condition")
axis[0,0].set_xlabel("Tube length -> m")
axis[0,0].ylabel("Temperature -> Deg C")

In [ ]:
result = subprocess.run(
    ["whb.exe"], capture_output=True, text=True
)
print("stdout:", result.stdout)
print("stderr:", result.stderr)

In [ ]:
fileno = 1
filename = "TEMPERATUREPROFILE" + str(fileno) + ".txt"
print(filename)

import shutil
shutil.copy("TEMPERATUREPROFILE.txt", filename)

In [ ]:
import subprocess
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import shutil
from scipy.interpolate import interp1d as ip

class whb:
    def __init__(self,ss,ts,k,temp,*args):
        self.ss = ss
        self.ts = ts
        self.k = k
        self.max_t = temp

        self.input_excel= "input_from_gips.xlsx"
        self.input_skel = "input_skeleton.txt"
        self.calcno = 123456
        self.command = "whb.exe"

        self.temp = []
        self.coat_len = []
        self.len = []

        self.fouling = [[0,0],[0,self.ts],[self.ss,0],[self.ss,self.ts]]
        self.fileno = [1,2,3,4]

        self.inp = "input.txt"
        
        if len(args)>1:
            if args[0] != "input_from_gips.xlsx":
                self.input_excel = args[0]
            if args[1] != "input_skeleton.txt":
                self.input_skel = args[1]
            if args[2] != "whb.exe":
                self.command = args[2]


    def create_input(self, fouling):
        dataframe = pd.read_excel(self.input_excel)
        dataframe['C'][0]

        with open(self.input_skel) as inp_skel:
            all_lines = inp_skel.readlines()

        inp = open("input.txt","w")

        print(all_lines[0], end = "",flush =True,file=inp)
        print(all_lines[1], end = "",flush =True,file=inp)
        print(all_lines[2], end = "",flush =True,file=inp)
        print(int(self.calcno), end = "\n",flush =True,file=inp)
        print(all_lines[3], end = "",flush =True,file=inp)
        print(dataframe['C'][1]/1000," ",dataframe['C'][2]/1000,file=inp)
        print(all_lines[4], end = "",flush =True,file=inp)
        print(dataframe['C'][3],file=inp)

        print(all_lines[5], end = "",flush =True,file=inp)
        print("20", end = "\n",flush =True,file=inp)
        print(all_lines[6], end = "",flush =True,file=inp)
        print("5", end = "\n",flush =True,file=inp)

        print(all_lines[7], end = "",flush =True,file=inp)
        print(int(dataframe['C'][4]),file=inp)

        print(all_lines[8], end = "",flush =True,file=inp)
        print(dataframe['C'][5],file=inp)

        print(all_lines[9], end = "",flush =True,file=inp)
        print(dataframe['C'][6]," ",dataframe['C'][7],file=inp)

        print(all_lines[10], end = "",flush =True,file=inp)
        print(dataframe['C'][8]," ",dataframe['C'][9],file=inp)

        print(all_lines[11], end = "",flush =True,file=inp)
        print(dataframe['C'][10]," ",dataframe['C'][11],file=inp)

        print(all_lines[12], end = "",flush =True,file=inp)
        print(dataframe['C'][12]," ",dataframe['C'][13],file=inp)

        print(all_lines[13], end = "",flush =True,file=inp)
        print(dataframe['C'][14],file=inp)

        print(all_lines[14], end = "",flush =True,file=inp)
        print(dataframe['C'][15],file=inp)

        print(all_lines[15], end = "",flush =True,file=inp)
        print(dataframe['C'][16],file=inp)

        print(all_lines[16], end = "",flush =True,file=inp)
        print(dataframe['C'][17],file=inp)

        print(all_lines[17], end = "",flush =True,file=inp)
        print(self.k, end = "\n",flush =True,file=inp)

        print(all_lines[18], end = "",flush =True,file=inp)
        print(fouling[0], end = "\n",flush =True,file=inp) # ss
        print(all_lines[19], end = "",flush =True,file=inp)
        print(fouling[1], end = "\n",flush =True,file=inp) # ts

        print(all_lines[20], end = "",flush =True,file=inp)
        print(all_lines[21], end = "",flush =True,file=inp)
        print(all_lines[22], end = "",flush =True,file=inp)
        print(all_lines[23], end = "",flush =True,file=inp)
        print(all_lines[24], end = "",flush =True,file=inp)

        inp.close()


    ''' the code below will execute and produce output txt '''
    def execute(self, fileno):
        popen = subprocess.Popen(self.command, stdout=subprocess.PIPE)
        lines_iterator = iter(popen.stdout.readline, b"")
        filename_out = "output" + str(fileno) + ".txt"
        file1 = open(filename_out,"w")
        while popen.poll() is None:
            for line in lines_iterator:
                nline = line.rstrip()
                print(nline.decode("latin"), end = "\r\n",flush =True, file=file1) # yield line
        file1.close()


    def create_data(self, fileno, fouling):
        filename_tem = "TEMPERATUREPROFILE" + str(fileno) + ".txt"
        shutil.copy("TEMPERATUREPROFILE.txt", filename_tem)
        with open(filename_tem) as f:
            lines = f.readlines()

        data = np.array(lines[2:])
        h = np.vectorize(lambda s: s.strip())(data)

        l=[]
        t=[]
        for i in h:
            j = i.split('        ')
            l.append(float(j[0]))
            t.append(float(j[1]))

        len = np.array(l, dtype=float)
        temp = np.array(t, dtype=float)

        coat_len = 0.0
        count = -1
        for i in temp:
            if i>=self.max_t:
                count = count + 1
        count = count + 1
        if count>0:
            coat_len = len[count]
            print("we need cladding upto %2.2f for shell side fouling: %1.7f and tube side fouling: %1.7f" %(len[count],fouling[0], fouling[1]))
        else:
            print("we dont need any coating for shell side fouling: %1.7f and tube side fouling: %1.7f \n" %(fouling[0], fouling[1])) 

        return len, temp, coat_len
    

    def plot_data(self, len, temp, fouling, coat_len):
        draw_line = False
        if np.amax(temp) >= self.max_t:
            fs = ip(temp, len, kind='cubic')
            len_at_400 = fs(self.max_t)
            print("Lenght of cladding at %3.2f temeprature is %2.2f \n" %(self.max_t, len_at_400))
            draw_line = True
        plt.plot(len, temp)
        label = "Temperature profile graph at U-shell = " + str(fouling[0]) + " and U-tube = " + str(fouling[1])
        plt.title(label)
        plt.xlabel("Tube length -> m")
        plt.ylabel("Temperature -> Deg C")
        if draw_line:
            plt.axhline(y=self.max_t, color="red", linestyle="-")
            plt.axvline(x=coat_len, color="green", linestyle=":")
            plt.axvline(x=len_at_400, color="blue", linestyle=":")
        plt.show()

    def do_stuff(self):
        for i in self.fileno:
            print("Case no --> %i \n" %i)
            self.create_input(self.fouling[i-1])
            self.execute(i)
            len, temp, coat_len = self.create_data(i, self.fouling[i-1])
            self.plot_data(len,temp,self.fouling[i-1],coat_len)
            print("\n-------------------------------------------------------------------")

    def updated_do_stuff(self):
        for i in self.fileno:
            print("Case no --> %i \n" %i)
            self.create_input(self.fouling[i-1])
            self.execute(i)
            len, temp, coat_len = self.create_data(i, self.fouling[i-1])
            self.len.append(len)
            self.temp.append(temp)
            self.coat_len.append(coat_len)

    def updated_plot_data(self):
        draw_line = False
        fig, axes = plt.subplots(2,2, figsize=(12,12))
        fig.text(0.5, 0.95, 'Temperature profile graph', ha='center', fontsize=20)
        fig.text(0.5, 0.05, 'Length [m]', ha='center', fontsize=15)
        fig.text(0.05, 0.5, 'Temperature [$\degree$C]', rotation=90, va='center', fontsize=15)
        count = 0
        print("\n\n-------- Interpolated outputs ------------ \n\n")
        for i in range(0,2):
            for j in range(0,2):
                draw_line = False
                if np.amax(self.temp[count]) >= self.max_t:
                    fs = ip(self.temp[count], self.len[count], kind='cubic')
                    len_at_400 = fs(self.max_t)
                    print("Lenght of cladding at %3.2f temeprature is %2.2f \n" %(self.max_t, len_at_400))
                    draw_line = True

                label = "$f_{shell}$ = " + str(self.fouling[count][0]) + " and $f_{tube}$ = " + str(self.fouling[count][1])

                axes[i][j].plot(self.len[count], self.temp[count], 'o-', label=label, ms=3)
                axes[i][j].legend()
                if draw_line:
                    s = '\n'.join((r'$T_{%1.1f} =  %1.2f m$' %(self.max_t, len_at_400), r'$T_{coat-len} = %1.2f m$' %(self.coat_len[count])))
                    axes[i][j].text(0.65,0.3,s,transform = axes[i][j].transAxes,fontsize=10, bbox=dict(facecolor='white', edgecolor='red'))
                    axes[i][j].axhline(y=self.max_t, color="red", linestyle="-")
                    axes[i][j].axvline(x=self.coat_len[count], color="green", linestyle=":")
                    axes[i][j].axvline(x=len_at_400, color="blue", linestyle=":")

                count = count + 1
        plt.show()

    def print_all(self):
        print(self.len)
        print(self.temp)
        print(self.coat_len)

In [ ]:
''' Testing purposes '''
ss=0.0001
ts=0.0005
k=31.0
max_t=400.0
my_whb = whb(ss,ts,k,max_t)
# my_whb.do_stuff()

my_whb.updated_do_stuff()
my_whb.print_all()
my_whb.updated_plot_data()